In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/goemotions/README.md
/kaggle/input/goemotions/goemotions_model_card.pdf
/kaggle/input/goemotions/GoEmotionsFormat.PNG
/kaggle/input/goemotions/replace_emotions.py
/kaggle/input/goemotions/extract_words.py
/kaggle/input/goemotions/calculate_metrics.py
/kaggle/input/goemotions/analyze_data.py
/kaggle/input/goemotions/tables/emotion_words.csv
/kaggle/input/goemotions/data/test.tsv
/kaggle/input/goemotions/data/ekman_mapping.json
/kaggle/input/goemotions/data/dev.tsv
/kaggle/input/goemotions/data/emotions.txt
/kaggle/input/goemotions/data/train.tsv
/kaggle/input/goemotions/data/ekman_labels.csv
/kaggle/input/goemotions/data/sentiment_dict.json
/kaggle/input/goemotions/data/sentiment_mapping.json
/kaggle/input/goemotions/data/full_dataset/goemotions_2.csv
/kaggle/input/goemotions/data/full_dataset/goemotions_1.csv
/kaggle/input/goemotions/data/full_dataset/goemotions_3.csv
/kaggle/input/goemotions/plots/hierarchical_corr.pdf
/kaggle/input/goemotions/plots/number_of_labels.pdf


In [2]:
import pandas as pd
import numpy as np
import torch

from torch.utils.data import TensorDataset, DataLoader
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification


2025-12-04 19:49:22.976839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764877763.164879      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764877763.218355      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
DATA_DIR = "/kaggle/input/goemotions/data"

train_df = pd.read_csv(f"{DATA_DIR}/train.tsv", sep="\t", header=None, names=["text","labels","id"])
dev_df   = pd.read_csv(f"{DATA_DIR}/dev.tsv", sep="\t", header=None, names=["text","labels","id"])
test_df  = pd.read_csv(f"{DATA_DIR}/test.tsv", sep="\t", header=None, names=["text","labels","id"])


In [4]:
NUM_LABELS = 28

def to_multihot(label_str):
    indices = list(map(int, label_str.split(",")))
    arr = np.zeros(NUM_LABELS)
    arr[indices] = 1
    return arr

train_df["multihot"] = train_df["labels"].apply(to_multihot)
dev_df["multihot"]   = dev_df["labels"].apply(to_multihot)
test_df["multihot"]  = test_df["labels"].apply(to_multihot)


In [5]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def encode(batch):
    return tokenizer(
        batch["text"].tolist(),
        truncation=True,
        padding="max_length",
        max_length=128
    )


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [6]:
train_enc = encode(train_df)
dev_enc   = encode(dev_df)
test_enc  = encode(test_df)


In [7]:
train_dataset = TensorDataset(
    torch.tensor(train_enc["input_ids"]),
    torch.tensor(train_enc["attention_mask"]),
    torch.tensor(np.vstack(train_df["multihot"].values), dtype=torch.float)
)

dev_dataset = TensorDataset(
    torch.tensor(dev_enc["input_ids"]),
    torch.tensor(dev_enc["attention_mask"]),
    torch.tensor(np.vstack(dev_df["multihot"].values), dtype=torch.float)
)

test_dataset = TensorDataset(
    torch.tensor(test_enc["input_ids"]),
    torch.tensor(test_enc["attention_mask"]),
    torch.tensor(np.vstack(test_df["multihot"].values), dtype=torch.float)
)


In [8]:
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader   = DataLoader(dev_dataset, batch_size=batch_size)


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification"
)

model.to(device)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=2e-5)

loss_fn = torch.nn.BCEWithLogitsLoss()


In [ ]:
from tqdm import tqdm

epochs = 2

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        input_ids, attention_masks, labels = [b.to(device) for b in batch]

        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_masks,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} Training Loss = {total_loss/len(train_loader)}")


In [ ]:
model.eval()
val_loss = 0

with torch.no_grad():
    for batch in tqdm(dev_loader, desc="Validation"):
        input_ids, attention_masks, labels = [b.to(device) for b in batch]

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_masks,
            labels=labels
        )

        val_loss += outputs.loss.item()

print("Validation Loss:", val_loss / len(dev_loader))


In [ ]:
SAVE_PATH = "/kaggle/working/emotion_model"

model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)

print("Model Saved →", SAVE_PATH)


In [ ]:
import shutil

shutil.make_archive("/kaggle/working/emotion_text_model", 'zip', "/kaggle/working/emotion_model")


**Test**

In [ ]:
import os

print(os.listdir("/kaggle/working"))
print(os.listdir("/kaggle/working/emotion_model"))
